#TODO
1. Zamiana sstas w KNN na cos sensowniejszego (done) zmieniełem na średnią
2. Zrobic KNN z kubełkami. (zrobione)
3. W np. logistic regression,knn dawać średnie zamiast losowych wartości.
4. wykresy 
5. ridge regression
6. random forest

In [3]:
# Please note that this code needs only to be run in a fresh runtime.
# However, it can be rerun afterwards too.
!pip install -q gdown httpimport
# Standard IPython notebook imports
import itertools
import io
import os
import numpy as np
import pandas as pd
import scipy.optimize as sopt
import scipy.stats as sstats
import seaborn as sns
import sklearn.ensemble
import sklearn.tree
from sklearn import datasets
from tqdm.auto import tqdm
import matplotlib.pyplot as pltdef
import httpimport

# In this way we can import functions straight from github
with httpimport.github_repo(
    "janchorowski", "nn_assignments", module="common", branch="nn18"
):
    from common.gradients import check_gradient
    from common.plotting import plot_mat

sns.set_style("whitegrid")


In [6]:

def KNN(train_X, train_Y, test_X, ks, verbose=False):
    """
    Compute predictions for various k
    Args:
        train_X: array of shape Ntrain x D
        train_Y: array of shape Ntrain
        test_X: array of shape Ntest x D
        ks: list of integers
    Returns:
        preds: dict k: predictions for k
    """
    # Cats data to float32
    train_X = train_X.astype(np.float32)
    test_X = test_X.astype(np.float32)

    # Alloc space for results
    preds = {}

    if verbose:
        print("Computing distances... ", end='')
    #
    # TODO: fill in an efficient distance matrix computation
    #    
    #dists = np.linalg.norm((train_X[:, None] - test_X), axis = -1)
    dists = np.sqrt((train_X**2).sum(1)[:,None] + (test_X**2).sum(1) - 2*train_X.dot(test_X.T))

    if verbose:
        print("Sorting... ", end='')
    
    # TODO: findes closest trainig points
    # Hint: use argsort
    closest = np.argsort(dists, axis = 0)

    if verbose:
        print("Computing predictions...", end='')
    
    targets = train_Y[closest]

    for k in ks:
        predictions = sstats.tmean(targets[:k])
        #Bierzemy średnią zamiast najczęściej występującej wartości
        preds[k] = predictions
    if verbose:
        print("Done")
    return preds


In [3]:
#from google.colab import drive
#drive.mount('/content/drive')

In [7]:
train_df = pd.read_csv("./petfinder-pawpularity-score/train.csv")
train_df["C"] = np.ones(9912)
test_df = pd.read_csv("./petfinder-pawpularity-score/test.csv")
test_df["Pawpularity"] = pd.read_csv("./petfinder-pawpularity-score/sample_submission.csv")["Pawpularity"]
test_df["C"] = np.ones(8)
test_df

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity,C
0,4128bae22183829d2b5fea10effdb0c3,1,0,1,0,0,1,1,0,0,1,0,1,67.75,1.0
1,43a2262d7738e3d420d453815151079e,0,1,0,0,0,0,1,1,0,0,0,0,59.15,1.0
2,4e429cead1848a298432a0acad014c9d,0,0,0,1,0,1,1,1,0,1,1,1,20.02,1.0
3,80bc3ccafcc51b66303c2c263aa38486,1,0,1,0,0,0,0,0,0,0,1,0,94.53,1.0
4,8f49844c382931444e68dffbe20228f4,1,1,1,0,1,1,0,1,0,1,1,0,89.82,1.0
5,b03f7041962238a7c9d6537e22f9b017,0,0,1,1,1,1,1,1,1,0,1,0,65.50,1.0
6,c978013571258ed6d4637f6e8cc9d6a3,1,0,0,0,1,1,0,1,0,1,1,1,71.42,1.0
7,e0de453c1bffc20c22b072b34b54e50f,1,0,1,0,0,0,0,0,1,0,0,1,5.85,1.0


In [8]:
def MSE(preds, test_Y):
    return(np.sqrt(np.mean((preds - test_Y)**2)))  #funkcja licząca wielkość błędu

KNN:

In [9]:
def KNN(train_X, train_Y, test_X, ks, verbose=False):
    """
    Compute predictions for various k
    Args:
        train_X: array of shape Ntrain x D
        train_Y: array of shape Ntrain
        test_X: array of shape Ntest x D
        ks: list of integers
    Returns:
        preds: dict k: predictions for k
    """
    # Cats data to float32
    train_X = train_X.astype(np.float32)
    test_X = test_X.astype(np.float32)

    # Alloc space for results
    preds = {}

    if verbose:
        print("Computing distances... ", end='')
    #
    # TODO: fill in an efficient distance matrix computation
    #    
    #dists = np.linalg.norm((train_X[:, None] - test_X), axis = -1)
    dists = np.sqrt((train_X**2).sum(1)[:,None] + (test_X**2).sum(1) - 2*train_X.dot(test_X.T))

    if verbose:
        print("Sorting... ", end='')
    
    # TODO: findes closest trainig points
    # Hint: use argsort
    closest = np.argsort(dists, axis = 0)

    if verbose:
        print("Computing predictions...", end='')
    
    targets = train_Y[closest]

    for k in ks:
        predictions = sstats.tmean(targets[:k])
        #Bierzemy średnią zamiast najczęściej występującej wartości
        preds[k] = predictions
    if verbose:
        print("Done")
    return preds

In [10]:
train_x = np.array(train_df[['Subject Focus', 'Eyes', "Face",	"Near",	"Action", "Accessory", "Group", "Collage", "Human", "Occlusion", "Info", "Blur", "C"]])
train_y = np.array(train_df["Pawpularity"])

test_x = np.array(test_df[['Subject Focus', 'Eyes', "Face",	"Near",	"Action", "Accessory", "Group", "Collage", "Human", "Occlusion", "Info", "Blur", "C"]])
test_y = np.array(test_df["Pawpularity"])

preds_KNN = KNN(train_x, train_y, test_x, [1, 3, 5, 7, 9, 11, 13, 15, 17, 19])

print(MSE(preds_KNN[1], test_y))
print(MSE(preds_KNN[3], test_y))
print(MSE(preds_KNN[5], test_y))
print(MSE(preds_KNN[7], test_y))
print(MSE(preds_KNN[9], test_y))
print(MSE(preds_KNN[11], test_y))
print(MSE(preds_KNN[13], test_y))
print(MSE(preds_KNN[15], test_y))
print(MSE(preds_KNN[17], test_y))
print(MSE(preds_KNN[19], test_y))
# wyniki dla 1,3,5 są lekko lepsze od zwykłej średniej
M = np.mean(train_y)

print(MSE(M, test_y)) #testujemy jaki byłby błąd gdybyśmy olali features i po prostu używali średniej wartości pawpularity

34.584439420062886
35.31583064922082
35.88227034344399
37.13190523289224
37.04147305702849
37.626997895941116
36.623213291233036
36.86968349621557
36.92476696470709
36.45848625605707
36.10768937034517


KNN cosine similarity:

In [11]:
def KNN_cos(train_X, train_Y, test_X, ks, verbose=False):
    """
    Compute predictions for various k
    Args:
        train_X: array of shape Ntrain x D
        train_Y: array of shape Ntrain
        test_X: array of shape Ntest x D
        ks: list of integers
    Returns:
        preds: dict k: predictions for k
    """
    # Cats data to float32
    train_X = train_X.astype(np.float32)
    test_X = test_X.astype(np.float32)

    # Alloc space for results
    preds = {}

    if verbose:
        print("Computing distances... ", end='')
    
    A_mA = train_X - train_X.mean(1)[:, None]
    B_mB = test_X - test_X.mean(1)[:, None]
    ssA = (A_mA**2).sum(1)
    ssB = (B_mB**2).sum(1)
    sims = np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))

    if verbose:
        print("Sorting... ", end='')
    
    # TODO: findes closest trainig points
    # Hint: use argsort
    closest = np.argsort(-sims, axis = 0)

    if verbose:
        print("Computing predictions...", end='')
    
    targets = train_Y[closest]

    for k in ks:
        #Bierzemy średnia zamiast mody
        predictions = sstats.tmean(targets[:k])
        preds[k] = predictions
    if verbose:
        print("Done")
    return preds

In [12]:
preds_KNN = KNN_cos(train_x, train_y, test_x,[1, 3, 5, 7, 9, 11, 13, 15, 17, 19])
print(MSE(preds_KNN[1], test_y))
print(MSE(preds_KNN[3], test_y))
print(MSE(preds_KNN[5], test_y))
print(MSE(preds_KNN[7], test_y))
print(MSE(preds_KNN[9], test_y))
print(MSE(preds_KNN[11], test_y))
print(MSE(preds_KNN[13], test_y))
print(MSE(preds_KNN[15], test_y))
print(MSE(preds_KNN[17], test_y))
print(MSE(preds_KNN[19], test_y))
#Wyniki dla 1,5,6,7,8 są lekko lepsze od normalnej średniej

35.339253316956196
37.18707369503548
36.95626232453709
36.56614384947377
35.543649278209266
35.294566511102126
35.11899240460627
35.6229255536375
36.204320529461675
36.138386312001494


Będziemy robili Knn z kubelkami <30,<60,<90

In [25]:
train_y_bucket = np.zeros(train_y.shape)
train_y_bucket[train_y <= 90] = 2
train_y_bucket[train_y <= 60] = 1
train_y_bucket[train_y <= 30] = 0
train_y_means=np.zeros(3)
train_y_means[0]=sstats.tmean(train_y,(0,30))
train_y_means[1]=sstats.tmean(train_y,(31,60))
train_y_means[2]=sstats.tmean(train_y,(61,90))
print(train_y_means)

[21.63369876 41.39272809 71.94222222]


In [33]:
def KNN_bucket(train_X, train_Y, test_X, ks, verbose=False):
    """
    Compute predictions for various k
    Args:
        train_X: array of shape Ntrain x D
        train_Y: array of shape Ntrain
        test_X: array of shape Ntest x D
        ks: list of integers
    Returns:
        preds: dict k: predictions for k
    """
    # Cats data to float32
    train_X = train_X.astype(np.float32)
    test_X = test_X.astype(np.float32)

    # Alloc space for results
    preds = {}

    if verbose:
        print("Computing distances... ", end='')
    #
    # TODO: fill in an efficient distance matrix computation
    #    
    #dists = np.linalg.norm((train_X[:, None] - test_X), axis = -1)
    dists = np.sqrt((train_X**2).sum(1)[:,None] + (test_X**2).sum(1) - 2*train_X.dot(test_X.T))

    if verbose:
        print("Sorting... ", end='')
    
    # TODO: findes closest trainig points
    # Hint: use argsort
    closest = np.argsort(dists, axis = 0)

    if verbose:
        print("Computing predictions...", end='')
    
    targets = train_Y[closest]

    for k in ks:
        predictions = sstats.mode(targets[:k])
        predictions = predictions[0].ravel()
        preds[k] = train_y_means[int(predictions[0])]
    if verbose:
        print("Done")
    return preds

In [34]:
train_x = np.array(train_df[['Subject Focus', 'Eyes', "Face",	"Near",	"Action", "Accessory", "Group", "Collage", "Human", "Occlusion", "Info", "Blur", "C"]])
train_y = np.array(train_df["Pawpularity"])

test_x = np.array(test_df[['Subject Focus', 'Eyes', "Face",	"Near",	"Action", "Accessory", "Group", "Collage", "Human", "Occlusion", "Info", "Blur", "C"]])
test_y = np.array(test_df["Pawpularity"])

preds_KNN = KNN_bucket(train_x, train_y_bucket, test_x, [1, 3, 5, 7, 9, 11, 13, 15, 17, 19])

print(MSE(preds_KNN[1], test_y))
print(MSE(preds_KNN[3], test_y))
print(MSE(preds_KNN[5], test_y))
print(MSE(preds_KNN[7], test_y))
print(MSE(preds_KNN[9], test_y))
print(MSE(preds_KNN[11], test_y))
print(MSE(preds_KNN[13], test_y))
print(MSE(preds_KNN[15], test_y))
print(MSE(preds_KNN[17], test_y))
print(MSE(preds_KNN[19], test_y))
# wyniki dla 1,2 są lekko lepsze od zwykłej średniej
M = np.mean(train_y)

print(MSE(M, test_y))

34.244841693979964
34.244841693979964
47.63413410503347
47.63413410503347
47.63413410503347
47.63413410503347
47.63413410503347
47.63413410503347
47.63413410503347
47.63413410503347
36.10768937034517


In [36]:
def KNN_bucket_cos(train_X, train_Y, test_X, ks, verbose=False):
    """
    Compute predictions for various k
    Args:
        train_X: array of shape Ntrain x D
        train_Y: array of shape Ntrain
        test_X: array of shape Ntest x D
        ks: list of integers
    Returns:
        preds: dict k: predictions for k
    """
    # Cats data to float32
    train_X = train_X.astype(np.float32)
    test_X = test_X.astype(np.float32)

    # Alloc space for results
    preds = {}

    if verbose:
        print("Computing distances... ", end='')
    
    A_mA = train_X - train_X.mean(1)[:, None]
    B_mB = test_X - test_X.mean(1)[:, None]
    ssA = (A_mA**2).sum(1)
    ssB = (B_mB**2).sum(1)
    sims = np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))

    if verbose:
        print("Sorting... ", end='')
    
    # TODO: findes closest trainig points
    # Hint: use argsort
    closest = np.argsort(-sims, axis = 0)

    if verbose:
        print("Computing predictions...", end='')
    
    targets = train_Y[closest]

    for k in ks:
        predictions = sstats.mode(targets[:k])
        predictions = predictions[0].ravel()
        preds[k] = train_y_means[int(predictions[0])]
    if verbose:
        print("Done")
    return preds

In [37]:
preds_KNN = KNN_bucket_cos(train_x, train_y_bucket, test_x,[1, 3, 5, 7, 9, 11, 13, 15, 17, 19])
print(MSE(preds_KNN[1], test_y))
print(MSE(preds_KNN[3], test_y))
print(MSE(preds_KNN[5], test_y))
print(MSE(preds_KNN[7], test_y))
print(MSE(preds_KNN[9], test_y))
print(MSE(preds_KNN[11], test_y))
print(MSE(preds_KNN[13], test_y))
print(MSE(preds_KNN[15], test_y))
print(MSE(preds_KNN[17], test_y))
print(MSE(preds_KNN[19], test_y))

34.244841693979964
47.63413410503347
47.63413410503347
47.63413410503347
47.63413410503347
47.63413410503347
47.63413410503347
47.63413410503347
47.63413410503347
47.63413410503347


logistic regression

In [13]:
def logreg_loss(Theta, X, Y):

    ThetaR = Theta.reshape(X.shape[1], 1)

    a = X @ ThetaR
    sigma = 1 / (1 + np.exp(-a))

    nll = -sum(Y * np.log(sigma) + (1 - Y) * np.log(1 - sigma))
    grad = X.T @ (sigma - Y)

    return nll, grad.reshape(Theta.shape)

train_y_logreg = np.zeros(train_y.shape)
#train_y_mean=sstats.tmean(train_y,(33,90))
#train_y_sum=np.sum(train_y)
train_y_logreg[train_y > 99] = 1
#klasyfikujemy jako 1 jeśli pawpularity jest duże, i jako 0 jeśli jest małe
train_y_logreg = train_y_logreg.reshape(-1,1)

Theta0 = np.ones((13,))
ThetaOpt = sopt.fmin_l_bfgs_b(
    lambda Theta: logreg_loss(Theta, train_x, train_y_logreg), np.array(Theta0)
)[0]
ThetaOpt


array([-0.49720214, -0.15903207, -0.00364125,  0.18000117, -0.35271007,
       -0.0865236 , -0.09552653,  0.03689894,  0.17321935, -0.16031754,
       -0.05671531, -0.26886885, -3.49277365])

In [14]:
h = test_x @ ThetaOpt #jeśli wynik większy bądź równy 0 to klasyfikujemy jako 1
h

array([-4.60485355, -3.71043332, -3.94382537, -4.05033235, -4.77201669,
       -3.69777095, -4.87821222, -4.08926654])

In [31]:
preds_logreg = np.full(test_y.shape, 60)
preds_logreg[h >= 0] = 100  #z tymi wartościami (20, 40) można pokombinować
preds_logreg

array([60, 60, 60, 60, 60, 60, 60, 60])

In [32]:
print(MSE(preds_logreg, test_y))

29.226759142949803


General Robust loss function

In [14]:
def rho(x, alpha, c): #the general robust loss function
    if alpha == 0 or alpha == 2:
        alpha = alpha + 0.000000001
    return(np.abs(alpha-2) / alpha * (((x/c)**2 / np.abs(alpha-2) + 1)**(alpha/2) - 1))

def d_rho(x, alpha, c): #and its derivative
    if alpha == 0 or alpha == 2:
        alpha = alpha + 0.000000001
    return(x / c**2 * ((x/c)**2 / np.abs(alpha-2) + 1)**(alpha/2 - 1))

In [15]:
def linreg_general_loss(Theta, X, Y, alpha, c):
    ThetaR = np.array(Theta)
    err = (Y - X @ ThetaR)
    loss = np.mean(rho(err, alpha, c))

    grad_err = d_rho(err, alpha, c)
    grad = np.array([np.mean(-grad_err * X[:, i]) for i in range(Theta.shape[0])])
    return loss, grad.reshape(Theta.shape)

In [16]:
train_y

array([63, 42, 28, ..., 20, 20, 30])

In [25]:
Theta0 = np.ones((13,))



for alpha in [-3, -2,-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12 ,13, 14,15,50]:
    for c in [1.]:
        ThetaOpt, _, dic = sopt.fmin_l_bfgs_b(
            lambda Theta: linreg_general_loss(Theta, train_x, train_y, alpha=alpha, c=c), np.array(Theta0)
        )
        pred = test_x @ ThetaOpt
        print(MSE(pred, test_y))


62.74256400711137
40.9578620760133
39.867510455636896
40.61718324999034
38.31332798393121
36.3820565240173
34.50314235794872
33.086588687333304
32.19123644602165
31.648816890675096
31.320061857510698
31.117219885880296
30.989092946005016
30.90461364066636
30.84725297305081
30.80632080252375
30.775910719879597
30.752623463080322
30.734115201836186
30.488868065952406
